In [1]:
import scala.io.Source
import scala.io.Codec

import org.apache.spark.rdd.RDD

In [2]:
val spark = SparkSession.builder()
  .master("local")
  .appName("Spark Scala Playground")
  .getOrCreate()
val sc = spark.sparkContext

spark = org.apache.spark.sql.SparkSession@75add937
sc = org.apache.spark.SparkContext@763cafd9


org.apache.spark.SparkContext@763cafd9

In [3]:
spark.version

3.3.0

In [4]:
val basePath = System.getProperty("user.dir") + "/data"

basePath = /mnt/project/data


/mnt/project/data

# Coursera Big Data Analysis with Scala and Spark

Objects/Classes from https://moocs.scala-lang.org/~dockermoocs/handouts/scala-3/wikipedia.zip, adapted for notbook

Data from https://moocs.scala-lang.org/~dockermoocs/bigdata/wikipedia-grading.dat


In [5]:
case class WikipediaArticle(title: String, text: String) {
  /**
    * @return Whether the text of this article mentions `lang` or not
    * @param lang Language to look for (e.g. "Scala")
    */
  def mentionsLanguage(lang: String): Boolean = text.split(' ').contains(lang)
}

defined class WikipediaArticle


In [6]:
object WikipediaData {

  def lines: List[String] = {
    val fileName = s"${basePath}/wikipedia-grading.dat"
    val source = Source.fromFile(fileName, Codec.UTF8.name)
    Source.fromFile(fileName).getLines().toList
  }

  def parse(line: String): WikipediaArticle = {
    val subs = "</title><text>"
    val i = line.indexOf(subs)
    val title = line.substring(14, i)
    val text  = line.substring(i + subs.length, line.length-16)
    WikipediaArticle(title, text)
  }
}

defined object WikipediaData


In [7]:
val wikiRdd: RDD[WikipediaArticle] = sc.parallelize(WikipediaData.lines.map(line => WikipediaData.parse(line)))

wikiRdd = ParallelCollectionRDD[0] at parallelize at <console>:30


ParallelCollectionRDD[0] at parallelize at <console>:30

In [8]:
wikiRdd.filter(article => article.mentionsLanguage("Scala")).count.intValue

24

# Spark Documentation Examples
https://spark.apache.org/docs/latest/sql-getting-started.html

In [9]:
val df = spark.read.json(s"${basePath}/people.json")
df.show()

df = [Name: string, age: bigint]


+-------+----+
|   Name| age|
+-------+----+
|Andreas|null|
|Herbert|  21|
| Norman|  29|
+-------+----+



[Name: string, age: bigint]

In [10]:
// This import is needed to use the $-notation
import spark.implicits._

In [11]:
// Print the schema in a tree format
df.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: long (nullable = true)



In [12]:
// Select only the "name" column
df.select("Name").show()

+-------+
|   Name|
+-------+
|Andreas|
|Herbert|
| Norman|
+-------+



In [13]:
// Select everybody, but increment the age by 1
df.select($"name", $"age" + 1).show()

+-------+---------+
|   name|(age + 1)|
+-------+---------+
|Andreas|     null|
|Herbert|       22|
| Norman|       30|
+-------+---------+



In [14]:
// Select people older than 21
df.filter($"age" > 21).show()

+------+---+
|  Name|age|
+------+---+
|Norman| 29|
+------+---+



In [15]:
// Count people by age
df.groupBy("age").count().show()

+----+-----+
| age|count|
+----+-----+
|  29|    1|
|null|    1|
|  21|    1|
+----+-----+

